# Titanic with MLP using Keras

## Setup

In [53]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import time

import pandas as pd
X = pd.read_csv('https://s3.amazonaws.com/jakechenawspublic/tutorials/sample_data/titanic/train.csv')
y = X.pop('Survived')

In [54]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [55]:
X.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

In [56]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [58]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False)

#from sklearn.preprocessing import OneHotEncoder
#enc = OneHotEncoder(sparse=False)

In [59]:
s_matrix = vec.fit_transform([{s:1} for s in X.pop('Sex')])
e_matrix = vec.fit_transform([{e:1} for e in X.pop('Embarked')])

In [60]:
print s_matrix.shape
print e_matrix.shape

(891, 2)
(891, 4)


In [64]:
frames = [X, pd.DataFrame(s_matrix, columns=['s0','s1']), pd.DataFrame(e_matrix, columns=['e0','e1','e2','e3'])]
X = pd.concat(frames, axis=1)

In [105]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,s0,s1,e0,e1,e2,e3
0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,1.0,0.0,0.0,1.0,0.0,0.0
2,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0
4,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0


In [65]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train2 = X_train.as_matrix()
X_valid2 = X_valid.as_matrix()

In [119]:
y_train2 = enc.fit_transform(y_train.values.reshape(-1,1))
y_valid2 = enc.fit_transform(y_valid.values.reshape(-1,1))

In [120]:
print X_train2.shape
print y_train2.shape

(596, 11)
(596, 2)


# Baseline: 0-layer Perceptron

## Define network

In [121]:
model = Sequential()
model.add(Dense(2, input_shape=(11,)))
model.add(Activation('softmax'))

## Train

In [122]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [123]:
t0 = time.time()
model.fit(X_train2, y_train2, batch_size=50, nb_epoch=100)
print "training time = {}".format(time.time() - t0)

Epoch 1/100
596/596 [==============================] - 0s - loss: 0.4054 - acc: 0.6275     
Epoch 2/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     
Epoch 3/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     
Epoch 4/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     
Epoch 5/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     
Epoch 6/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     

## Evaluate

In [95]:
score = model.evaluate(X_valid2, y_valid2, batch_size=50, verbose=1)
print '\n {}'.format(score)

 50/295 [====>.........................] - ETA: 0s
 score is [1.1920929171853875e-07, 0.59322032888056864]


# 2-layer MLP

## Define network

In [130]:
model = Sequential()
model.add(Dense(100, input_shape=(11,)))
model.add(Activation('tanh'))
model.add(Dense(200))
model.add(Activation('tanh'))
model.add(Dense(2))
model.add(Activation('softmax'))

## Train

In [131]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [132]:
t0 = time.time()
model.fit(X_train2, y_train2, batch_size=50, nb_epoch=100)
print "training time = {}".format(time.time() - t0)

Epoch 1/100
596/596 [==============================] - 0s - loss: 0.0718 - acc: 0.6107     
Epoch 2/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     
Epoch 3/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     
Epoch 4/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     
Epoch 5/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     
Epoch 6/100
596/596 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.6275     

## Evaluate

In [103]:
score = model.evaluate(X_valid2, y_valid2, batch_size=50, verbose=1)
print '\n {}'.format(score)

 50/295 [====>.........................] - ETA: 0s
 [1.1920929171853875e-07, 0.59322032888056864]
